## Pytorch Tutorial - Fashion MNIST

In [1]:
import torch
from torch.utils.data import DataLoader
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
learning_rate = 0.0005
training_epochs = 16
batch_size = 64

In [4]:
mnist_train = dsets.FashionMNIST(root='FashionMNIST/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.FashionMNIST(root='FashionMNIST/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

In [5]:
len(mnist_train)

60000

In [6]:
# # 전체 데이터 dataloader 생성시
# data_loader = DataLoader(dataset=mnist_train,
#                         batch_size=batch_size,
#                         shuffle=True,
#                         drop_last=True) #drop last ~ batch 단위로 데이터를 불러올 때 마지막 batch의 크기가 다를 경우 나머지의 해당 배치 데이터를 drop하는 option

In [7]:
from torch.utils.data.dataset import random_split
train_dataset, val_dataset = random_split(mnist_train, [50000,10000])

In [8]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

In [10]:
class CNN_basic(torch.nn.Module):
    def __init__(self):
        super(CNN_basic, self).__init__()
        self.keep_prob = 0.5 # drop out ratio
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 64, kernel_size=3, stride=1 , padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 64, kernel_size=3, stride=1 , padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1 , padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        )
        
        self.fc1 = torch.nn.Linear(4*4*128, 256, bias = True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1-self.keep_prob)
        )
        
        self.fc2 = torch.nn.Linear(256, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1) #Flatten for fully connected layer
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [11]:
model_ft = CNN_basic().to(device)

In [12]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model_ft.parameters(), lr = learning_rate)

In [13]:
total_batch = len(train_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 781


In [14]:
len(val_loader)

157

In [15]:
train_loss_history = [] 
val_loss_history = []

for epoch in range(training_epochs):
    avg_cost = 0
    
    for j, (X, y) in enumerate(train_loader):
        X= X.to(device)
        y= y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model_ft(X)
        cost = criterion(hypothesis, y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
        if j%100 == 0:
            print("epoch {} / batch {} / loss {}".format(epoch+1, j, cost.data))
    train_loss_history.append(avg_cost)
    
    with torch.no_grad():
        val_loss = 0.0
        for X_val, y_val in val_loader:
            X_val= X_val.to(device)
            y_val = y_val.to(device)
            
            y_val_pred = model_ft(X_val)
            v_loss = criterion(y_val_pred, y_val)
            val_loss += v_loss / len(val_loader)
        val_loss_history.append(val_loss)
        print("validation loss {}".format(val_loss))
        print('=====================================')
        
    model_ft.train()    

epoch 1 / batch 0 / loss 2.293971061706543
epoch 1 / batch 100 / loss 0.9591710567474365
epoch 1 / batch 200 / loss 0.6048954725265503
epoch 1 / batch 300 / loss 0.38689643144607544
epoch 1 / batch 400 / loss 0.4305373728275299
epoch 1 / batch 500 / loss 0.40433090925216675
epoch 1 / batch 600 / loss 0.2554273307323456
epoch 1 / batch 700 / loss 0.5498675107955933
validation loss 0.4058389961719513
epoch 2 / batch 0 / loss 0.5841600298881531
epoch 2 / batch 100 / loss 0.23906071484088898
epoch 2 / batch 200 / loss 0.23192040622234344
epoch 2 / batch 300 / loss 0.4296759366989136
epoch 2 / batch 400 / loss 0.40734267234802246
epoch 2 / batch 500 / loss 0.35348930954933167
epoch 2 / batch 600 / loss 0.35246437788009644
epoch 2 / batch 700 / loss 0.2690242826938629
validation loss 0.3379691541194916
epoch 3 / batch 0 / loss 0.435079425573349
epoch 3 / batch 100 / loss 0.2619270980358124
epoch 3 / batch 200 / loss 0.23060572147369385
epoch 3 / batch 300 / loss 0.22528234124183655
epoch 3 /

In [16]:
model_ft.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (imgs, labels) in enumerate(test_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model_ft(imgs)
        _, argmax = torch.max(outputs, 1)
        total += imgs.size(0)
        correct += (labels == argmax).sum().item()
    
    print('Accuracy for {} images: {:.2f}%'.format(total, correct / total * 100))     

Accuracy for 10000 images: 92.14%
